<a href="https://colab.research.google.com/github/Ibrahim-Ezmazy/python-Projects/blob/main/GoogleNet_CoffeeDiseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score


In [3]:
# Mount Google Drive if needed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

# Define image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Path to your dataset folders
train_data_dir = '/content/drive/MyDrive/training'
val_data_dir = '/content/drive/MyDrive/validation'


In [6]:

# Define constants
batch_size = 20
num_classes = 2
epochs = 10

# Create image generators
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(150, 150),  # GoogleNet input size
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

# Load GoogleNet model
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add custom layers on top of GoogleNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Found 419 images belonging to 2 classes.
Found 111 images belonging to 2 classes.
87910968/87910968 [==============================] - 1s 0us/step


In [7]:

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/10
20/20 [==============================] - 182s 9s/step - loss: 0.5186 - accuracy: 0.7243 - val_loss: 0.3549 - val_accuracy: 0.8600
Epoch 2/10
20/20 [==============================] - 75s 4s/step - loss: 0.1964 - accuracy: 0.9223 - val_loss: 0.1713 - val_accuracy: 0.9500
Epoch 3/10
20/20 [==============================] - 76s 4s/step - loss: 0.1145 - accuracy: 0.9649 - val_loss: 0.1644 - val_accuracy: 0.9600
Epoch 4/10
20/20 [==============================] - 76s 4s/step - loss: 0.0907 - accuracy: 0.9825 - val_loss: 0.1427 - val_accuracy: 0.9600
Epoch 5/10
20/20 [==============================] - 71s 4s/step - loss: 0.0719 - accuracy: 0.9875 - val_loss: 0.1277 - val_accuracy: 0.9700
Epoch 6/10
20/20 [==============================] - 74s 4s/step - loss: 0.0614 - accuracy: 0.9875 - val_loss: 0.1381 - val_accuracy: 0.9400
Epoch 7/10
20/20 [==============================] - 80s 4s/step - loss: 0.0626 - accuracy: 0.9825 - val_loss: 0.1251 - val_accuracy: 0.9600
Epoch 8/10
20/20 [=

In [8]:

# Evaluate the model
evaluation = model.evaluate(validation_generator)
print("Test Accuracy:", evaluation[1])

# Calculate confusion matrix
validation_generator.reset()  # Reset the generator to avoid shuffling issues
y_pred = model.predict(validation_generator)
y_true = validation_generator.classes
y_pred_classes = tf.argmax(y_pred, axis=1)
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate precision, recall, and F-score
precision = precision_score(y_true, y_pred_classes)
recall = recall_score(y_true, y_pred_classes)
f_score = f1_score(y_true, y_pred_classes)
print("Precision:", precision)
print("Recall:", recall)
print("F-score:", f_score)


6/6 [==============================] - 16s 3s/step - loss: 0.1123 - accuracy: 0.9640
Test Accuracy: 0.9639639854431152
6/6 [==============================] - 18s 3s/step
Confusion Matrix:
[[22 26]
 [26 37]]
Precision: 0.5873015873015873
Recall: 0.5873015873015873
F-score: 0.5873015873015873
